In [1]:
import numpy as np
# from datetime import datetime

def WaterBalance_MonthlyFlow(Dates, Climatedata, AnalogYr, ACOND):
    
    Array_len = len(Climatedata)

    #####
    # Alternative flow estimation approach using KNN unsupervised learning; NOTE: Not implemented in Python version
    # 1 = on
    # 0 = off
    KNN_switch = 0
    #
    # Check the specification of row = 10....

    # if KNN_switch == 1:
    #     BT_UCFlow, BT_HistFlow = KNN_Model(varargin) 

    # Extract the date information from the input climate data 
    # Year, Month, _ = np.split(Dates, 3, axis=1)
    # Extract the number of days in a month from the date information
    # Days = np.array([np.datetime64(f'{y}-{m:02}') for y, m in zip(Year, Month)]).astype('datetime64[M]').astype(int) - np.array([np.datetime64(f'{y}-{m:02}') for y, m in zip(Year, Month)]).astype('datetime64[M-1]').astype(int)

    days_per_month = np.array([31,28,31,30,31,30,31,31,30,31,30,31])
    Year = np.array([datetime.strptime(date, '%Y-%m-%d').year for date in Dates])
    Month = np.array([datetime.strptime(date, '%Y-%m-%d').month for date in Dates])
    Days = days_per_month[Month-1]

    # Adjust 'Days' for leap years
    inds_leap_years = np.argwhere(
        (np.mod(Year,4) == 0) &  # leap years are divisible by 4
        (Month == 2) &           # only want indices of February
        (Year != 2100)           # 2100 is not a leap year
    ).squeeze()
    Days[inds_leap_years] = 29   # 29 days in each leap year February
    
    # Calculate the water year form the date information
    WaterYear = np.where(Month > 9, Year + 1, Year)
    
    Precip = np.zeros((Array_len, 4))
    Precip[:, 2] = np.round(Climatedata[:, 0])

    ##### NOTE: MATLAB rounds decimals of 0.5 up to the nearest integer, while Python rounds decimals 
    ##### of 0.5 to the nearest EVEN integer.  This causes small discrepancies between the MATLAB and
    ##### Python versions of this code.  These differences are small in terms of precipitation (a maximum
    ##### difference of 1mm per month); however, this 1mm can be consequential.  In select circumstances,
    ##### it is enough to cause the MATLAB flow to be greater than the minimum required flow, while the 
    ##### Python flow is less than the minimum required flow.  This can cause different pathways to be
    ##### followed in the nested conditional statements.  For this reason, I will implement the below 
    ##### modification to the Python rounding such that results are exactly the same as the MATLAB version.

    ##### START ROUNDING MODIFICATION
    for kk in range(len(Climatedata[:,0])):
        rounded_down = np.floor(Climatedata[kk,0])
        if Climatedata[kk,0] - rounded_down == 0.5:
            if np.mod(Climatedata[kk,0] - 0.5, 2) == 0: # if the lower nearby integer is even, i.e. if 0.5 will round down
                Precip[kk,2] += 1 #add 1 to the rounded-down precip
    ##### END ROUNDING MODIFICATION

    # CHECK THE INPUT FILES FOR THE CORRECT COLUMN
    
    MaxT_c = Climatedata[:, 1]
    MinT_c = Climatedata[:, 2]

    # Precip in mm / month
    Precip_mmmonth = Precip[:, 2] 

    # Average temperatures in degC
    AT = MaxT_c + MinT_c
    Avg_T = AT / 2

    # Variables
    ETo_1 = 0.75
    ETo_2 = 0.50
    Recharge = 0.48   # Recharge proportion
    c_base = 0.095    # Baseflow coefficient
    d_base = 0.000175 # Drought baseflow
    Area = 2955110400 # Drainage area in sq. ft

    # Santa Cruz Mountains are around 37deg North; Months 1-12
    Pref = np.array([[1, 0.226],
                     [2, 0.246],
                     [3, 0.27],
                     [4, 0.294],
                     [5, 0.314],
                     [6, 0.328],
                     [7, 0.324],
                     [8, 0.304],
                     [9, 0.28],
                     [10, 0.25],
                     [11, 0.226],
                     [12, 0.216]])

    #----------Calculations below----------#

    # p constant - lookup from P_ref (P reference table)
    Month_col = 0
    Pref_col = 0
    Pref_value = 1
    # loc = np.where(Month == Pref[:, Pref_col])[0]
    # p = Pref[loc, Pref_value]
    loc = np.zeros(len(Month), dtype = 'int')
    for kk_month in range(12):
        inds = np.where(Month == Pref[:,Pref_col][kk_month])[0]
        loc[inds] = kk_month
    p = Pref[loc, Pref_value]

    # ETo (mm/day) = p(ETo1*T+ETo2)
    ETo = p * (ETo_1 * Avg_T + ETo_2)
    ETo_ftday = ETo / 1000 * 3.28094 #unit conversion
    
    # Precip mm/year (sum for water year)
    # Initialize the Precip_mmyr variable for filling
    Precip_mmyr = np.zeros((Array_len, 2))
    
    Year_min = np.min(Year)
    Year_max = np.max(Year)
    
    for yr in range(Year_min, Year_max + 1): # 2019 to 2070
        rowNo = np.where(WaterYear == yr)[0]
        # Psum_len = len(rowNo)
        minR = np.min(rowNo)
        maxR = np.max(rowNo)
        Psum = np.sum(Precip_mmmonth[minR:maxR+1]) # Sum for the whole water year
        Precip_mmyr[rowNo, 0] = WaterYear[rowNo]
        Precip_mmyr[rowNo, 1] = Psum

    # Recharge mm / year
    Recharge_mmyr = Precip_mmyr[:, 1] * Recharge

    # Precip (ft/day)
    Precip_ftday = (Precip_mmmonth / 1000) * 3.28084 / Days
    AvgP = np.mean(Precip_ftday)

    # Initialize the Re variable for filling
    # Array_len = len(Precip)
    Re = np.zeros(Array_len)
    
    for row in range(Array_len):
        
        if Precip[row, 2] < 10:
            
            Re[row] = 0
            
        elif Precip[row, 2] >= 10:
            
            Re[row] = (Precip[row, 2] / Precip_mmyr[row, 1]) * ((Recharge_mmyr[row] / 2.65 / 30.48) / 365)

    # Soil moisture factor
    # Initialize the soil variable for filling
    Soil = np.zeros(Array_len)
    
    for row in range(9):
        Soil[row] = np.nan
    
    for row in range(9, Array_len):
        minR = row - 9
        maxR = row
        Soil[row] = np.mean(Precip_ftday[minR:maxR+1]) / AvgP

    # Baseflow ft/day
    # Initialize the baseflow variable for filling
    Baseflow = np.zeros(Array_len)
    
    for row in range(9):
        Baseflow[row] = np.nan
    
    for row in range(9, Array_len):
        minR = row - 6
        maxR = row
        Baseflow[row] = d_base + np.sum(Precip_ftday[minR:maxR+1]) * Soil[row] * c_base

    # Uncalibrated modelled flow (ft/day) = Precip + Baseflow - Re - ETo
    mflow = Precip_ftday + Baseflow - Re - ETo_ftday
    mflow_cfs = mflow * Area / 86400 * Days # unit conversion from ft/day to cfs/day

    # Calibrated model flow (cfs/day)
    Cali_flow = np.zeros(Array_len)
    
    for row in range(9):
        Cali_flow[row] = np.nan

    # Calculate the calibrated monthly flow at Big Trees. This could be done
    # dynamically using any of a type of regression functions. Here it is hard
    # coded to be simple. This should be updated in the future and can
    # easily be done.
    
    for row in range(9, Array_len):
        
        if KNN_switch == 1:
            
            if mflow_cfs[row] <= -10000:
                
                Cali_flow[row] = 1069.98628 * np.exp(0.00009 * mflow_cfs[row])
                
            elif mflow_cfs[row] > -10000:
                
                pass
                
        else:
            
            if mflow_cfs[row] <= 4000:
                
                Cali_flow[row] = 1069.98628 * np.exp(0.00009 * mflow_cfs[row])
                
            elif mflow_cfs[row] > 4000:
                
                Cali_flow[row] = (0.000005 * mflow_cfs[row] ** 2) + (0.1055 * mflow_cfs[row] + 996.03)

    # Build the final matrix
    BT_MonthlyFlow = np.zeros((Array_len, 6))
    # Fill the matrix
    BT_MonthlyFlow[:, 0] = Days
    BT_MonthlyFlow[:, 1] = Month
    BT_MonthlyFlow[:, 2] = Year
    BT_MonthlyFlow[:, 3] = WaterYear
    BT_MonthlyFlow[:, 4] = Cali_flow
    
    CummFlow = np.zeros(Array_len)
    
    for j in range(9, Array_len):
        if BT_MonthlyFlow[j, 1] == 10:
            CummFlow[j] = BT_MonthlyFlow[j, 4]
        else:
            CummFlow[j] = BT_MonthlyFlow[j, 4] + CummFlow[j - 1]
    
    BT_MonthlyFlow[:, 5] = CummFlow

    # Build an array of annual to flow in cfs-days for use in the 
    # Hydrologic Conditions and daily streamflow calculations

    Years = np.max(BT_MonthlyFlow[:, 3]) - np.min(BT_MonthlyFlow[:, 3]) + 1

    BT_AnnFlow = np.zeros((int(Years), 2))
    
    counter = 0
    
    for j in range(Array_len): # Go through each month of cumulative flow
        
        if BT_MonthlyFlow[j, 1] == 9: #9: # If September, i.e. if the end of a water year
            
            counter += 1 # Increase the count
            
            if j == 8: #9: # If 9th index, i.e. if in the first 9 months of data (Jan through Sept 1937) 
                
                BT_AnnFlow[j - (8 * counter), 0] = BT_MonthlyFlow[j, 3] #(8*counter)
                BT_AnnFlow[j - (8 * counter), 1] = BT_MonthlyFlow[j, 5] #(8*counter)
                
            else:

                # Just to assign values to the correct row
                idx = (j - (11 * (counter - 1))) - 8
                BT_AnnFlow[idx, 0] = BT_MonthlyFlow[j, 3]
                BT_AnnFlow[idx, 1] = BT_MonthlyFlow[j, 5]
    
    # Build a precip matrix for daily flow calculations later
    # Precip = np.zeros((Array_len,4)) ##### FUTURE SAM: THIS MIGHT NEED TO BE UNBOLDED
    Precip[:, 0] = BT_MonthlyFlow[:, 2]
    Precip[:, 1] = BT_MonthlyFlow[:, 1]
    
    if ACOND == 1:
        idx = np.where(WaterYear == 1937)[0][-1] # Use 2019 for future years and 1937 for historical
        idx1 = np.where(BT_AnnFlow == 1937)[0][-1] # Use 2019 for future years and 1937 for historical
    else:
        idx = np.where(WaterYear == 2022)[0][-1] # Use 2019 for future years, 2015 for the catalog and 1937 for historical
        idx1 = np.where(BT_AnnFlow == 2022)[0][-1] # Use 2019 for future years, 2015 for the catalog and 1937 for historical

        ##### EDIT TO MATCH NEW MATLAB CODE: Change year to 2019 -- consistent with comments, and with CC_Daily_DateArr_3.txt
        idx = np.where(WaterYear == 2019)[0][-1] # Use 2019 for future years, 2015 for the catalog and 1937 for historical
        idx1 = np.where(BT_AnnFlow == 2019)[0][-1] # Use 2019 for future years, 2015 for the catalog and 1937 for historical
    
    BT_MonthlyFlow = BT_MonthlyFlow[idx+1:]
    BT_AnnFlow = BT_AnnFlow[idx1+1:]
    Precip = Precip[idx+1:]
    AnalogYr = AnalogYr[idx+1:]

    # Now go through and sum precip over each water year
    # Now go through and sum the Cali_flow for each water year
    for j in range(len(Precip)):
        if Precip[j, 1] == 10:
            Precip[j, 3] = Precip[j, 2]
        else:
            Precip[j, 3] = Precip[j - 1, 3] + Precip[j, 2]
    
    return BT_MonthlyFlow, BT_AnnFlow, Precip, AnalogYr